In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np

import time

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstarping_partitions_full_int_q,
    )

2024-07-28 17:36:51.349502: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:36:51.636832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 17:36:51.636957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 17:36:51.691882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 17:36:51.815611: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:36:51.816587: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-28 17:36:56.144132: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 17:36:56.144854: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2024-07-28 17:36:56.377452: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 17:36:56.450106: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

In [6]:
BASE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_final.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_31 (InputLayer)       [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1d_88 (Conv1D)          (None, 47996, 32)            192       ['input_31[0][0]']            
                                                                                                  
 max_pooling1d_30 (MaxPooli  (None, 2999, 32)             0         ['conv1d_88[0][0]']           
 ng1D)                                                                                            
                                                                                                  
 dropout_116 (Dropout)       (None, 2999, 32)             0         ['max_pooling1d_30[0][0

### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpqdjs1kmn/assets


INFO:tensorflow:Assets written to: /tmp/tmpqdjs1kmn/assets
2024-07-28 17:40:13.994816: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 17:40:13.994920: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 17:40:13.996539: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpqdjs1kmn
2024-07-28 17:40:13.999290: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 17:40:13.999324: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpqdjs1kmn
2024-07-28 17:40:14.006315: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-28 17:40:14.009048: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 17:40:14.103475: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [8]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9492930304912707

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9423526984523124

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.005 seconds
Max: 0.009 seconds
Min: 0.004 seconds




In [9]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 43.238 Kilobytes


### Dynamic range quantization

In [10]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpozh5_uou/assets


INFO:tensorflow:Assets written to: /tmp/tmpozh5_uou/assets
2024-07-28 17:41:39.595638: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 17:41:39.595715: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 17:41:39.595907: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpozh5_uou
2024-07-28 17:41:39.599603: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 17:41:39.599640: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpozh5_uou
2024-07-28 17:41:39.607982: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 17:41:39.695891: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpozh5_uou
2024-07-28 17:41:39.726060: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9455744835578986

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9483671127612608

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.005 seconds
Max: 0.016 seconds
M

In [12]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 26.188 Kilobytes


### Float16 quantization

In [13]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpq08hv4aj/assets


INFO:tensorflow:Assets written to: /tmp/tmpq08hv4aj/assets
2024-07-28 17:43:06.872786: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 17:43:06.872861: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 17:43:06.873043: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpq08hv4aj
2024-07-28 17:43:06.875926: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 17:43:06.875961: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpq08hv4aj
2024-07-28 17:43:06.884330: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 17:43:06.970996: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpq08hv4aj
2024-07-28 17:43:06.997405: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9474939742098535

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9398628116277996

Time for Test dataset:
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Time to make a prediction for a single data point
Mean: 0.005 seconds
Max: 0.011 seconds
M

In [15]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 33.258 Kilobytes


### Float fallback

In [16]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3l3w4ef8/assets


INFO:tensorflow:Assets written to: /tmp/tmp3l3w4ef8/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 17:44:33.768569: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 17:44:33.768621: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 17:44:33.768800: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp3l3w4ef8
2024-07-28 17:44:33.771563: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 17:44:33.771599: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp3l3w4ef8
2024-07-28 17:44:33.779189: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 17:44

In [17]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 81.96%
Recall: 49.58%
Precision: 95.51%
F1-score: 65.27%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.6472534462186863

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.6538850882601293

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 82.91%
Recall: 48.91%
Precision: 98.25%
F1-score: 65.31%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.654022465395096

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.6577044423934139

Time for Test dataset:
Accuracy: 82.91%
Recall: 48.91%
Precision: 98.25%
F1-score: 65.31%

Time to make a prediction for a single data point
Mean: 0.007 seconds
Max: 0.033 seconds
M

### Full integer quantization

In [18]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmptpo9xp6k/assets


INFO:tensorflow:Assets written to: /tmp/tmptpo9xp6k/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 17:46:55.148322: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 17:46:55.148413: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 17:46:55.148634: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmptpo9xp6k
2024-07-28 17:46:55.151187: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 17:46:55.151212: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmptpo9xp6k
2024-07-28 17:46:55.161322: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 17:46

In [19]:
interpreter = tf.lite.Interpreter(model_content=full_int_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [20]:
import pathlib

tflite_models_dir = pathlib.Path("../time_series_models_from_notebooks/tiny_transformer/quantized/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_q_file = tflite_models_dir/"tiny_transformer_time_series_full_int_q.tflite"
tflite_model_full_int_q_file.write_bytes(full_int_model)

30056

In [21]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_q_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_q_file), "KB")

Validation dataset:
Accuracy: 68.77%
Recall: 9.53%
Precision: 91.84%
F1-score: 17.27%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.17077708333937164

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.16754854035334962

Test dataset:
Accuracy: 71.86%
Recall: 14.63%
Precision: 98.53%
F1-score: 25.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.2517551253995913

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.25365593872024567

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.007 seconds
Max: 0.011 seconds
Min: 0.006 seconds
File size: 29.352 Kilobytes
